## Data Retrieval Notebook

In this notebook, we use code provided by the NREL to request and handle the necessary data and metadata from the NOW-23 Great Lakes dataset. The code provided by the NREL has been modified to make the automatic querying and concatenation of data possible.

In [ ]:
import requests
import pandas as pd
import urllib.parse
import time
import os

API_KEY = os.read(os.open(r"API_KEY.txt", os.O_RDONLY), 100)
EMAIL = "alexanderwycoff@icloud.com"

In [ ]:
# The function used to generate the 100 random site ids: 
# np.random.randint(0, 388080, (100))

site_ids = pd.read_csv("site ids", index_col=0)
site_ids = [str(site_ids.iloc[x, 0]) for x in range(100)]

In [ ]:
BASE_URL = "https://developer.nrel.gov/api/wind-toolkit/v2/wind/offshore-great-lakes-download.csv?"
POINTS = site_ids

# This code is provided by NREL to request and handle data
# Some modifications have been made so that csv requests can be made and appended automatically

def main():
    input_data = {
        'attributes': 'temperature_80m,turbulent_kinetic_energy_80m,winddirection_80m,windspeed_80m',
        'interval': '60',

        'api_key': API_KEY,
        'email': EMAIL,
    }
    
    for id, location_ids in enumerate(POINTS):

        print(f'Making request for point group {id + 1} of {len(POINTS)}...')
        for name in ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']:
            
            input_data['names'] = name
            input_data['location_ids'] = location_ids

            if '.csv' in BASE_URL:
                url = BASE_URL + urllib.parse.urlencode(input_data, True)
                data = pd.read_csv(url, on_bad_lines='skip', skiprows=2)
                
                print(f'Response data for year {name} and site {location_ids}')

                file_loc = r"Data\NOW-23 Great Lakes [2000-2020] 60min/" + str(location_ids + ".csv")
                
                if name == '2000':
                    data.to_csv(file_loc)
                else:
                    pd.concat([pd.read_csv(file_loc, index_col=0), data]).to_csv(file_loc)
            else:
                headers = {
                    'x-api-key': API_KEY
                }
                data = get_response_json_and_handle_errors(requests.post(BASE_URL, input_data, headers=headers))
                download_url = data['outputs']['downloadUrl']
                # You can do with what you will the download url
                print(data['outputs']['message'])
                print(f"Data can be downloaded from this url when ready: {download_url}")

                data = pd.read_csv(download_url)

                # Delay for 1 second to prevent rate limiting
                time.sleep(1)
            print(f'Processed')


def get_response_json_and_handle_errors(response: requests.Response) -> dict:
    """Takes the given response and handles any errors, along with providing
    the resulting json

    Parameters
    ----------
    response : requests.Response
        The response object

    Returns
    -------
    dict
        The resulting json
    """
    if response.status_code != 200:
        print(f"An error has occurred with the server or the request. The request response code/status: {response.status_code} {response.reason}")
        print(f"The response body: {response.text}")
        exit(1)

    try:
        response_json = response.json()
    except:
        print(f"The response couldn't be parsed as JSON, likely an issue with the server, here is the text: {response.text}")
        exit(1)

    if len(response_json['errors']) > 0:
        errors = '\n'.join(response_json['errors'])
        print(f"The request errored out, here are the errors: {errors}")
        exit(1)
    return response_json

if __name__ == "__main__":
    main()

In [ ]:
# This code is modified from the above cell to retrieve metadata for all randomly selected points and concatenate it

BASE_URL = "https://developer.nrel.gov/api/wind-toolkit/v2/wind/offshore-great-lakes-download.csv?"
POINTS = site_ids

input_data = {
    'attributes': 'temperature_80m,turbulent_kinetic_energy_80m,winddirection_80m,windspeed_80m',
    'interval': '60',

    'api_key': API_KEY,
    'email': EMAIL,
}

for id, location_ids in enumerate(POINTS):

    print(f'Making request for point group {id + 1} of {len(POINTS)}...')
    for name in ['2000']:

        input_data['names'] = name
        input_data['location_ids'] = location_ids

        if '.csv' in BASE_URL:
            url = BASE_URL + urllib.parse.urlencode(input_data, True)
            data = pd.read_csv(url, on_bad_lines='skip')

            print(f'Response metadata for year {name} and site {location_ids}')
            
            file_loc = r"Data\NOW-23 Great Lakes [2000-2020] metadata.csv"
            if id == 0:
                data.to_csv(file_loc)
            else:
                pd.concat([pd.read_csv(file_loc, index_col=0), data]).to_csv(file_loc)